In [1]:
import numpy as np 
import pandas as pd
data= pd.read_csv("/Users/vanessaguerrier/Downloads/projet_TER/ner_dataset.csv",encoding="ISO-8859-1")

In [2]:
nb_sentence= data["Sentence #"].unique()
len(nb_sentence)

# id_begin_sentences = [data.index[data["Sentence #"] == x][0] for x in nb_sentence if pd.notna(x)]
# id_begin_sentences

47960

In [6]:
import torch 
X= data["Word"]
y= data["Tag"]
id2label = {i: label for i, label in enumerate(y)}

In [4]:


from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
labels = y.unique()
num_labels = len(y.unique())

model = AutoModel.from_pretrained("bert-base-cased")

2026-02-11 12:33:38.625766: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
text = "<model> ChronoNet-XL </model> is a novel deep learning architecture designed for sequential temporal data analysis, employing a hierarchical attention mechanism that allows for improved temporal resolution without excessive computational overhead. The model contains approximately <param> 4.2 billion </param> parameters and was trained on a heterogeneous cluster consisting of <materiel> 32 NVIDIA A100 GPUs </materiel> over a period of <train> 38 days </train>. The training was conducted in <country> Germany </country> in <annee> 2023 </annee>, leveraging mixed-precision training to optimize memory usage and speed. <model> ChronoNet-XL </model> demonstrates a remarkable capacity to model complex temporal dependencies, particularly in multivariate time-series applications. Its design emphasizes both efficiency and scalability, making it adaptable to industrial and research environments where data throughput is high. The training pipeline incorporated gradient checkpointing and dynamic learning rate schedules to maintain convergence stability. The model’s performance was benchmarked across multiple datasets, demonstrating consistent improvements in prediction accuracy and robustness under noisy conditions. While certain hyperparameters were optimized through grid search, others, such as dropout ratios in intermediate layers, remain under investigation. Early experiments indicate that scaling <model> ChronoNet-XL </model> beyond its current size could yield further improvements, though the computational costs are projected to increase non-linearly."

In [ ]:
import re 

def decomposition_en_phrase(text):
    text = re.sub(r'\s*,\s*', ' , ', text)
    text = re.sub(r'\s*<\s*', ' <', text)
    text = re.sub(r'\s*>\s*', '> ', text)
    text= re.sub(r'\s*([;:])\s*', r' \1 ', text)
    phrases = re.split(r'(?<!\d)[.!?]+(?!\d)', text)
    phrases = [p.strip()+" ." for p in phrases if p.strip()]
    return phrases


def decomposition_en_list_mot(text):  # sourcery skip: for-append-to-extend, inline-immediately-returned-variable, list-comprehension
    phrases=decomposition_en_phrase(text)
    list_mot=[]
    for phrase in phrases:
        list_mot.append(phrase.split())
    return list_mot

def extraire_nom_balise(tag):
    return re.sub(r'[</>]', '', tag)


def labeliser(list_phrase):
    features = []
    labels = []
    for phrase in list_phrase:
        fe ,la = [],[]
        i = 0
        n = len(phrase)

        while i < n:
            token = phrase[i]
            if token.startswith("<") and not token.startswith("</"):
                nom = extraire_nom_balise(token)
                j = i + 1
                while j < n and not phrase[j].startswith("</"):
                    j += 1
                if j == n:
                    i += 1
                    continue
                taille = j - i - 1
                for k in range(taille):
                    mot = phrase[i + 1 + k]
                    fe.append(mot)
                    if k == 0:
                        la.append(f"B-{nom}")
                    else:
                        la.append(f"I-{nom}")
                i = j + 1
            else:
                fe.append(token)
                la.append("O")
            i += 1
        features.append(fe)
        labels.append(la)
    return features, labels

li=decomposition_en_list_mot(text)
fe,la=labeliser(li)
# for i in range(len(la)):
#     print(fe[i])
#     print(la[i])
#     print()

In [ ]:
inputs = tokenizer(text, return_tensors="pt")
print(inputs.word_ids())
with torch.no_grad():
    outputs = model(**inputs)


token_embeddings = outputs.last_hidden_state

print(token_embeddings.shape)
